In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df=pd.read_csv("data.csv", sep = ";")
df

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    df.drop('Target', axis=1),
    df['Target'],
    test_size=(1.0/3), random_state=42)

## Weight and Bias (W&B)

Weight and Bias es una plataforma que facilita el seguimiento, visualización y colaboración en experimentos de aprendizaje automático. Permite registrar métricas, gráficos y modelos de manera centralizada para facilitar el análisis y la comprensión de los resultados obtenidos durante el entrenamiento de modelos.

**Funcionalidades Principales**

1. **Seguimiento de Experimentos:** Permite registrar métricas clave como pérdida y precisión a lo largo del entrenamiento para cada experimento.

2. **Visualización Interactiva:** Proporciona gráficos dinámicos para explorar la evolución de métricas y comparar diferentes experimentos de manera intuitiva.

3. **Colaboración y Reproducibilidad:** Facilita compartir resultados con colegas y mantener un registro detallado de todos los experimentos realizados.

> Se requiere una cuenta de W&B, asi como crear un team/equipo y un proyecto para poder utilizar la plataforma. Para más información, visite: https://wandb.ai/site

In [ ]:
!pip install wandb

In [ ]:
import wandb

wandb.login()

### Sweeps en Weight and Bias

Uno de los aspectos destacados de Weight and Bias son los "sweeps". Estos permiten explorar múltiples combinaciones de hiperparámetros de manera automática, registrando y comparando los resultados de cada configuración. Algunas características de los sweeps incluyen:

- **Automatización de Experimentos:** Ejecución paralela de múltiples configuraciones de hiperparámetros para optimizar el rendimiento del modelo.

- **Análisis de Resultados:** Visualización automática de los resultados de cada configuración para identificar la mejor combinación de hiperparámetros.

- **Ajuste Eficiente:** Permite ajustar rápidamente los hiperparámetros sin necesidad de intervención manual intensiva.

Más información sobre Weight and Bias en la documentación oficial: https://docs.wandb.ai/guides/sweeps

In [ ]:
WANDB_TEAM_NAME = "kidnixt-ort"
WANDB_PROJECT = "arboles-decisión"

# La configuración del sweep: los parámetros que queremos optimizar
sweep_config = {
    "name": "students-dropout-sweep",
    "method": "random",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "parameters": {
        "max_depth": {"values": [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]},
        "criterion": {"values": ["gini", "entropy"]},
    },
}

# Crea un nuevo sweep (podríamos usar un sweep existente si ya lo hubiéramos creado)
sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT)

### Hacer una nueva division del dataset Dev en train+val

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train,
    Y_train,
    test_size=(1.0/3), random_state=42)

In [ ]:

from joblib import dump, load

def wand_log(depth, train_loss, val_loss):
    wandb.log({"depth": depth, "train_loss": train_loss, "val_loss": val_loss})


def sweep_run():
    """
    Función que se ejecutará en cada run del sweep.
    """
    # inicializar un nuevo run
    wandb.init()
    # leer la configuración del run
    config = wandb.config
    run_max_depth = config.max_depth
    run_criterion = config.criterion

    arbol = DecisionTreeClassifier(criterion = run_criterion, max_depth=run_max_depth)
    arbol.fit(X_train, Y_train)

    # log
    wand_log(run_max_depth, 1- arbol.score(X_train, Y_train), 1- arbol.score(X_val, Y_val))

    #save model
    dump(arbol, 'tree.joblib')
    wandb.save("tree.joblib")

    wandb.finish()

### Corremos el Sweep

Para ejecutar un sweep, creamos un agente que le pide a W&B un conjunto de hiperparámetros para cada experimento. Luego, ejecutamos cada experimento con esos hiperparámetros y registramos los resultados en W&B.

Es importante que podemos ejecutar en varios entornos, incluso al mismo tiempo, para explorar diferentes configuraciones de hiperparámetros y comparar los resultados.

In [ ]:
wandb.agent(sweep_id, function=sweep_run, count=5)

### Seleccion del mejor modelo

In [ ]:
api = wandb.Api()

# nos traemos el sweep (objeto) para analizar los resultados
sweep = api.sweep(f"{WANDB_TEAM_NAME}/{WANDB_PROJECT}/{sweep_id}")

# obtenemos el mejor run
best_run = sweep.best_run()

# imprimimos el mejor run
print(f"Best run {best_run.name} with {best_run.summary['val_loss']}")

# descargamos el modelo del mejor run
best_run.file("tree.joblib").download(replace=True)

### Evaluacion final con el modelo seleccionado

In [ ]:
clf = load('tree.joblib')

# Evaluamos el modelo en el conjunto de test
test_loss = 1 - clf.score(X_test, Y_test)

print(f"Test Loss: {test_loss:.5f}")

## Sweep de curva LOSS en validation (demora en ejecutar)

In [ ]:
# Configuración del sweep
sweep_config = {
    "name": "students-dropout-sweep",
    "method": "grid",  # Puede ser 'grid', 'random', etc.
    "metric": {
        "name": "val_loss",
        "goal": "minimize"
    },
    "parameters": {
        "max_depth": {
            "values": [3, 5, 10, 15]
        },
        "min_samples_split": {
            "values": [2, 5, 10]
        },
        "min_samples_leaf": {
            "values": [1, 2, 4]
        },
        "criterion": {
            "values": ["gini"]
        }
    }

}

#notar que cambiamos el nombre del project para que no se sobreescriba el anterior
WANDB_PROJECT = "arboles-decisión-curva"

sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT)

X_train, X_val, Y_train, Y_val = train_test_split(
    X_train,
    Y_train,
    test_size=(1.0/3), random_state=42)

#redefinimos sweep_run para que tome en cuenta los nuevos parámetros
def sweep_run():
    """
    Función que se ejecutará en cada run del sweep.
    """
    # inicializar un nuevo run
    wandb.init(settings=wandb.Settings(silent=True)) ## silenciar la salida
    # leer la configuración del run
    config = wandb.config
    run_max_depth = config.max_depth
    run_criterion = config.criterion
    run_min_samples_split = config.min_samples_split
    run_min_samples_leaf = config.min_samples_leaf

    arbol = DecisionTreeClassifier(criterion = run_criterion, 
                                   max_depth=run_max_depth, 
                                   min_samples_split=run_min_samples_split, 
                                   min_samples_leaf=run_min_samples_leaf)
    arbol.fit(X_train, Y_train)

    # log
    train_loss = 1- arbol.score(X_train, Y_train)
    val_loss = 1- arbol.score(X_val, Y_val)
    wandb.log({
        "train_loss": train_loss,
        "val_loss": val_loss
    })

    #save model
    dump(arbol, 'tree-curve.joblib')
    wandb.save("tree-curve.joblib")

    wandb.finish()

wandb.agent(sweep_id, function=sweep_run)



### Cargamos el mejor modelo y calculamos LOSS en test

In [ ]:
api = wandb.Api()

# nos traemos el sweep (objeto) para analizar los resultados
sweep = api.sweep(f"{WANDB_TEAM_NAME}/{WANDB_PROJECT}/{sweep_id}")

# obtenemos el mejor run
best_run = sweep.best_run()

# imprimimos el mejor run
print(f"Best run {best_run.name} with {best_run.summary['val_loss']}")

# descargamos el modelo del mejor run
best_run.file("tree-curve.joblib").download(replace=True)

In [ ]:
clf = load('tree.joblib')

# Evaluamos el modelo en el conjunto de test
test_loss = 1 - clf.score(X_test, Y_test)

print(f"Test Loss: {test_loss:.5f}")